In [ ]:
import pandas as pd
import numpy as np

# Load datasets (adjust paths to where you saved them)
orders = pd.read_csv("olist_orders_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")
items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")

In [ ]:
# Look at first few rows
print(orders.head())

# Check for missing values
print(orders.isnull().sum())

# Check data types
print(orders.dtypes)

In [ ]:
# Example: Fill missing delivery dates with a placeholder (NaT for datetime)
orders['order_delivered_customer_date'] = pd.to_datetime(
    orders['order_delivered_customer_date'], errors='coerce'
)
orders['order_delivered_customer_date'].fillna(pd.NaT, inplace=True)

# Payment dataset: check for missing payments
payments = payments.dropna()  # or fill with 0 if appropriate

In [ ]:
# Orders should have unique order_id
print(orders.duplicated(subset=['order_id']).sum())

orders = orders.drop_duplicates(subset=['order_id'])

In [ ]:
# Convert date columns to datetime
date_cols = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Convert numeric columns properly
payments['payment_value'] = payments['payment_value'].astype(float)

In [ ]:
# Join orders with customers
orders_customers = pd.merge(
    orders, customers,
    on="customer_id", how="left"
)

# Join with payments
orders_full = pd.merge(
    orders_customers, payments,
    on="order_id", how="left"
)

print(orders_full.head())

In [ ]:
# Create new features

# Delivery time
orders_full['delivery_time_days'] = (
    orders_full['order_delivered_customer_date'] - orders_full['order_purchase_timestamp']
).dt.days

# Late delivery flag
orders_full['late_delivery'] = (
    orders_full['order_delivered_customer_date'] > orders_full['order_estimated_delivery_date']
)

# Group payments per order (some have multiple installments)
order_values = payments.groupby('order_id')['payment_value'].sum().reset_index()
order_values.rename(columns={'payment_value': 'total_order_value'}, inplace=True)

# Merge back into full dataset
orders_full = pd.merge(orders_full, order_values, on='order_id', how='left')

In [ ]:
# Exploratory Data Analysis

import matplotlib.pyplot as plt
import seaborn as sns

# Set a style
sns.set(style="whitegrid")

In [ ]:
# Sales Trends – How much revenue is the store making?

# Convert order purchase timestamp to month
orders_full['order_month'] = orders_full['order_purchase_timestamp'].dt.to_period("M")

# Revenue per month
monthly_revenue = (
    orders_full.groupby('order_month')['total_order_value']
    .sum()
    .reset_index()
)

# Plot
plt.figure(figsize=(12,5))
sns.lineplot(x='order_month', y='total_order_value', data=monthly_revenue, marker="o")
plt.title("Monthly Revenue Over Time")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Delivery Performance – What’s the distribution of delivery times, and how often are they late?

# Delivery time distribution
plt.figure(figsize=(10,5))
sns.histplot(orders_full['delivery_time_days'], bins=30, kde=True)
plt.title("Distribution of Delivery Time (days)")
plt.xlabel("Days")
plt.show()

# Late delivery rate
late_rate = orders_full['late_delivery'].mean() * 100
print(f"Late deliveries: {late_rate:.2f}%")

In [ ]:
# Customer segmentation – Where are customers concentrated?

# Orders per customer
orders_per_customer = (
    orders_full.groupby('customer_id')['order_id']
    .nunique()
    .reset_index(name="num_orders")
)

plt.figure(figsize=(8,5))
sns.histplot(orders_per_customer['num_orders'], bins=20, kde=False)
plt.title("Orders per Customer")
plt.xlabel("Number of Orders")
plt.ylabel("Count of Customers")
plt.show()

print(orders_per_customer['num_orders'].describe())